### Set Path (Won't be needed once `setup.py` is finished)

In [1]:
import sys
sys.path.insert(0, sys.path[0][:-8])

In [2]:
import torch
from tqdm import tqdm
from torch.autograd import Variable
from sklearn.metrics import mean_absolute_error

### Auglichem imports

In [3]:
from auglichem.crystal import Compose, RandomRotationTransformation, SupercellTransformation
from auglichem.crystal.data import CrystalDatasetWrapper
from auglichem.crystal.models import CrystalGraphConvNet as CGCNN

### Set up dataset

In [4]:
#help(CrystalDatasetWrapper)

In [5]:
#help(CrystalDatasetWrapper.__init__)

In [6]:
# Create transformation
transform = [
    SupercellTransformation(),
    RandomRotationTransformation([1,0,0], 45)
]

# Initialize dataset object
dataset = CrystalDatasetWrapper("lanthanides", batch_size=256, kfolds=2,
                                valid_size=0.1, test_size=0.1)

# Get train/valid/test splits as loaders
train_loader, valid_loader = dataset.get_data_loaders(transform=transform, fold=0)

Ignoring splitting. Using pre-split k folds.


  1%|▎                                                             | 21/4166 [00:00<03:21, 20.54it/s]/Users/clo/miniforge3/envs/auglichem_3.8/lib/python3.8/site-packages/pymatgen/io/cif.py:1165: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))
100%|████████████████████████████████████████████████████████████| 4166/4166 [00:44<00:00, 93.01it/s]


### Initialize model with task from data

In [7]:
# Get model
structures, _, _ = dataset[0]
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]

model = CGCNN(orig_atom_fea_len, nbr_fea_len)

### Initialize traning loop

In [8]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

### Train the model

In [9]:
for epoch in range(1):
    for bn, (data, target, _) in tqdm(enumerate(train_loader)):
        optimizer.zero_grad()
        input_var = (Variable(data[0]),
                     Variable(data[1]),
                     data[2],
                     data[3])
        
        pred = model(*input_var)
        loss = criterion(pred, target)
        
        loss.backward()
        optimizer.step()

24it [02:01,  5.06s/it]


### Test the model

In [10]:
def evaluate(model, valid_loader):
    with torch.no_grad():
        model.eval()
        data, target, _ = next(iter(valid_loader))
        input_var = (Variable(data[0]),
                     Variable(data[1]),
                     data[2],
                     data[3])

        pred = model(*input_var)
        mae = mean_absolute_error(pred, target)   
        
    print("TEST MAE: {0:.3f}".format(loss.detach()))

In [11]:
evaluate(model, valid_loader)

TEST MAE: 0.080


### Model saving/loading example

In [12]:
# Save model
torch.save(model.state_dict(), "./saved_models/example_cgcnn")

In [13]:
# Instantiate new model and evaluate
structures, _, _ = dataset[0]
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]

model = CGCNN(orig_atom_fea_len, nbr_fea_len)

evaluate(model, valid_loader)

TEST MAE: 0.080


In [14]:
# Load saved model and evaluate
model.load_state_dict(torch.load("./saved_models/example_cgcnn"))
evaluate(model, valid_loader)

TEST MAE: 0.080
